In [1]:
import pandas as pd
from sklearn.cluster import MeanShift
import math 

import warnings
warnings.filterwarnings('ignore')

In [2]:
init_data = pd.read_csv('C:\\workspace\\phystech\\SBT_ML\\umn_foursquare_datasets\\checkins.dat', sep='|')
init_data.columns = init_data.columns.map(lambda x: x.strip())
init_data.latitude = init_data.latitude.map(lambda x: str(x).strip())
init_data.longitude = init_data.longitude.map(lambda x: str(x).strip())
init_data = init_data[(init_data.latitude.map(lambda x: len(x)) != 0)&(init_data.longitude.map(lambda x: len(x)) != 0)]

init_data.drop(init_data.head(1).index, inplace=True)
init_data.drop(init_data.tail(1).index, inplace=True)

init_data.latitude = init_data.latitude.map(lambda x: float(x))
init_data.longitude = init_data.longitude.map(lambda x: float(x))

In [3]:
init_data = init_data[:100000]

In [4]:
clusters_labels = MeanShift(bandwidth=.1, n_jobs=-1).fit_predict(init_data[['latitude', 'longitude']])

In [5]:
print len(clusters_labels)

100000


In [6]:
init_data.loc[:,'clust_label'] = clusters_labels
clust_counts = dict(init_data.clust_label.value_counts())
big_clusters = [i[0] for i in filter(lambda x: x[1]>15, clust_counts.items())]

In [7]:
cutt_data = init_data[init_data.clust_label.isin(big_clusters)]

In [8]:
# clust_df = pd.DataFrame(columns=['cluster', 'cl_latitude', 'cl_longitude'])
rows_list = []
for clust in cutt_data.clust_label.unique():
    rows_list.append({'cluster':clust,
                      'cl_latitude':cutt_data[cutt_data.clust_label == clust].latitude.mean(),
                      'cl_longitude':cutt_data[cutt_data.clust_label == clust].longitude.mean()})
clust_df = pd.DataFrame(rows_list)

In [9]:
def geo_dist(shir_1, dolg_1, shir_2, dolg_2):
    # rad - радиус сферы (Земли)
    rad = 6372795

    #в радианах
    lat1 = shir_1*math.pi/180.
    lat2 = shir_2*math.pi/180.
    long1 = dolg_1*math.pi/180.
    long2 = dolg_2*math.pi/180.

    #косинусы и синусы широт и разницы долгот
    cl1 = math.cos(lat1)
    cl2 = math.cos(lat2)
    sl1 = math.sin(lat1)
    sl2 = math.sin(lat2)
    delta = long2 - long1
    cdelta = math.cos(delta)
    sdelta = math.sin(delta)

    #вычисления длины большого круга
    y = math.sqrt(math.pow(cl2*sdelta,2)+math.pow(cl1*sl2-sl1*cl2*cdelta,2))
    x = sl1*sl2+cl1*cl2*cdelta
    ad = math.atan2(y,x)
    dist = ad*rad
    
    return dist

In [10]:
nearest_offices = [(33.751277, -118.188740), (25.867736, -80.324116), (51.503016, -0.075479), 
                   (52.378894, 4.885084), (39.366487, 117.036146), (-33.868457, 151.205134)]
dist_to_nearest = []
for clust in clust_df.cluster:
    distances = []
    for office in nearest_offices:
        distances.append(geo_dist(office[0], office[1], clust_df[clust_df.cluster==clust].iloc[0].cl_latitude, 
                                  clust_df[clust_df.cluster==clust].iloc[0].cl_longitude))
    dist_to_nearest.append(min(distances))
clust_df.loc[:,'dist_to_nearest'] = dist_to_nearest

In [11]:
result_df = clust_df.sort_values(['dist_to_nearest']).head(20)[['cl_latitude', 'cl_longitude']]
result_df.columns = ['latitude', 'longitude']
result_df

,latitude,longitude
271,52.372964,4.892317
140,-33.860630,151.204776
355,25.845672,-80.318891
119,51.502786,-0.124192
18,33.805774,-118.155494
166,25.790049,-80.212558
185,33.864529,-118.038644
141,25.698997,-80.286142
113,26.006447,-80.201624
94,33.864732,-118.360905


In [12]:
result_df.to_csv("C:\\workspace\\phystech\\SBT_ML\\clust_result.csv", index=False)